In [ ]:
 //#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
using NSE_SIMPLE;
using NSE_SIMPLE.LowMach;
using ilPSP.LinSolvers.PARDISO;
Init();

In [ ]:
using BoSSS.Solution.Queries;

In [ ]:
static string name = "TAYLORGREENVORTEX_TEMPORALCONV_STUDY";

In [ ]:
BoSSSshell.WorkflowMgm.Init(name);

In [ ]:
static var myBatch = GetDefaultQueue();
// (myBatch as MsHPC2012Client).ComputeNodes = new string[]{"HPCLUSTER3"};
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

In [ ]:
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

## Temporal convergence study

In [ ]:
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:

int[] bdfOrders = new int[]{1,2,3,4};
// double[] dts = new double[]{1/1,1.0/2,1.0/10,1.0/20,1.0/100};
double[] dts = new double[]{1/1,1.0/2,1.0/10,1.0/20};


int[] dgDegree = new int[]{5};
int[] res = new int[]{30};
double finaltime = 3;



In [ ]:
Type solverXNSEC = typeof(BoSSS.Application.XNSEC.XNSEC);


In [ ]:
for (int dg = 0; dg < dgDegree.Length; dg++) {
    for (int r = 0; r < res.Length; r++) {
        for (int bdf = 0; bdf < bdfOrders.Length; bdf++) {
            for (int dt = 0; dt < dts.Length; dt++) {
                 string ending = "P" + dgDegree[dg] + "K" + res[r] + "dt" + dts[dt] + "bdf" + bdfOrders[bdf];
                 string jobName = string.Format("XNSEC_TaylorGreenVortex" + ending);
                 string StartupString = string.Format("cs:BoSSS.Application.XNSEC.FullNSEControlExamples.UnsteadyTaylorVortex({0}, {1}, {2}, {3}, @\"{4}\", {5})", dgDegree[dg], res[r], dts[dt],bdfOrders[bdf],myDb.Path, finaltime);
                 var oneJob = new Job(jobName, solverXNSEC);
                 oneJob.SetControlStatement(StartupString);
                 oneJob.NumberOfMPIProcs = 4;
                 oneJob.NumberOfThreads = 1;
                 oneJob.Activate(myBatch);
            }
        }
    }
}

In [ ]:
// wait for all jobs to finish (up to 1 day, check every 2 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;

In [ ]:
dts

In [ ]:
// int[] bdfOrders = new int[]{1,2,3,4};
// double[] dts = new double[]{1/1,1.0/2,1.0/10,1.0/20,1.0/100};

MultidimensionalArray errors_u = MultidimensionalArray.Create(1, bdfOrders.Length, dts.Length);
MultidimensionalArray errors_v = MultidimensionalArray.Create(1, bdfOrders.Length, dts.Length);
MultidimensionalArray errors_p = MultidimensionalArray.Create(1, bdfOrders.Length, dts.Length);



for (int b = 0; b < bdfOrders.Length; b++) {
    for (int dt = 0; dt < dts.Length; dt++) {
        var sess_b = myDb.Sessions.Where(sess => ((int)(long)(sess.KeysAndQueries["TimeSteppingScheme"]) == bdfOrders[b]) && ((double)(sess.KeysAndQueries["dtFixed"]) == dts[dt])).Single(); // Searchs for sessions with BDF order b

        try {
            errors_u[0, b, dt] = (double)sess_b.KeysAndQueries["SolL2err_u"];
        } catch {

        }


        try {
            errors_v[0, b, dt] = (double)sess_b.KeysAndQueries["SolL2err_v"];
             } catch {

        }


        try {
            errors_p[0, b, dt] = (double)sess_b.KeysAndQueries["SolL2err_p"];
        } catch {

        }

    }
}


In [ ]:

var dtsInverse = new double[dts.Length];

for(int i = 0; i < dts.Length; i++){
    dtsInverse[i] = 1.0/dts[i];
}

In [ ]:
dtsInverse

In [ ]:
errors_u.ExtractSubArrayShallow(0,0,-1).To1DArray()

In [ ]:
errors_u.ExtractSubArrayShallow(0,1,-1).To1DArray()

In [ ]:
errors_u.ExtractSubArrayShallow(0,2,-1).To1DArray()

In [ ]:
errors_u.ExtractSubArrayShallow(0,3,-1).To1DArray()

In [ ]:
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;
LineColors[] allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
PointTypes[] myPointTypes = new PointTypes[] { PointTypes.Diamond, PointTypes.Box, PointTypes.LowerTriangle, PointTypes.OpenLowerTriangle, };
LineColors[] myCollors = new LineColors[] { LineColors.Red, LineColors.Orange, LineColors.Blue,  LineColors.Black };

var plotcf = new Plot2Ddata();
plotcf.LogX = true;
plotcf.LogY = true;
for(int i= 0 ; i < bdfOrders.Length ; i++ ){
    fmt.LineColor = myCollors[i];
    plotcf.AddDataGroup("bdf=" + bdfOrders[i], dtsInverse, errors_u.ExtractSubArrayShallow(0,i,-1).To1DArray(), fmt);
}

var gp = plotcf.ToGnuplot();
gp.PlotSVG(xRes:1000,yRes:400)


In [ ]:
Tab = Tab.ExtractColumns("DGdegree:Velocity*","DGdegree:Temperature", "DGdegree:Pressure","Grid:hMin","SolL2err_u","SolL2err_v","SolL2err_T","SolL2err_p");

var ErrorPlot_u = Tab. ToPlot ("Grid:hMin", "SolL2err_u","DGdegree:Velocity*"); // column for grou
var ErrorPlot_v = Tab. ToPlot ("Grid:hMin", "SolL2err_v","DGdegree:Velocity*"); // column for grou
var ErrorPlot_T = Tab. ToPlot ("Grid:hMin", "SolL2err_T","DGdegree:Temperature"); // column for grou
var ErrorPlot_p = Tab. ToPlot ("Grid:hMin", "SolL2err_p","DGdegree:Pressure"); // column for grou
ErrorPlot_u.LogX = true ;
ErrorPlot_u.LogY = true ;

ErrorPlot_v.LogX = true ;
ErrorPlot_v.LogY = true ;

ErrorPlot_T.LogX = true ;
ErrorPlot_T.LogY = true ;

ErrorPlot_p.LogX = true ;
ErrorPlot_p.LogY = true ;

In [ ]:

Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 2];
PlotTable[0,0] = ErrorPlot_u;
PlotTable[0,1] = ErrorPlot_v;
PlotTable[1,0] = ErrorPlot_T;
PlotTable[1,1] = ErrorPlot_p;
var p = PlotTable.ToGnuplot();
p.PlotNow()

In [ ]:
// // string dataPathResultsBase = @"C:\Users\Gutierrez\Documents\data\convergenceProblemData\" ;
// string dataPathResultsBase = @"D:\ThesisDGCombustion\dgcombustionthesis\data\ConvStudy_Couette";
// string folderpath          = dataPathResultsBase ;// + ending +"LowTemp"+ @"\";
// System.IO.Directory.CreateDirectory(folderpath); // Create folder for storing data

// string[] varnames = new string[]{ "Pressure" };// 
// // string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure" };// 

// foreach (var varname in varnames) {
//     foreach (int pDeg in dgDegree) {
//        var pDegSessions = myDb.Sessions.Where(
//             Si => (Convert.ToInt32(Si.KeysAndQueries["DGdegree:Pressure"]) == pDeg-1) 
//          ).ToArray();


//         Plot2Ddata pDegPlot =
//          pDegSessions.ToEstimatedGridConvergenceData(varname,
//            xAxis_Is_hOrDof: true, // false selects DOFs for x-axis 
//            normType: NormType.L2_embedded );


//         pDegPlot.SaveTextFileToPublish(folderpath+"\\"+  varname + "DG" + pDeg, false);
//         Console.WriteLine(folderpath+"\\"+  varname + "DG" + pDeg);
//     }
// }
